# Class : De Bruijn graph

---
## Before Class
1. Review slides on De Bruijn graphs and Eulerian walks

---
## Learning Objectives
1. Understand and implement De Bruijn graphs for assembly


---
## De Bruijn graphs

In class today we will be implementing one of the primary assembly algorithms from short-read data that is used today. We will implement a simple form of the algorithm where we assume perfect sequencing. That is, everything is sequenced exactly once and there are no errors or variants in the sequencing. This graph will use a similar structure to our previous work with trees, but we will need to track edges between nodes in our graph. We have provided the basic class structure as well as functions to `add_edge` and `remove_edge` from the graph. 

```
build_debruijn_graph:
define substring length k and input string
For each k-length substring of input:
  split k mer into left and right k-1 mer
  add k-1 mers as nodes with a directed edge from left k-1 mer to right k-1 mer
```

---
## Eulerian walk

For the second part of the implementation, we will use our De Bruijn graph to output a valid sequence from the assembly. This is implemented as a recursive algorithm by considering all valid edges. You will notice that as you change k, we are able to better recapitulate our sequence depending on how repetitive it is. In a more complex implementation of a Eulerian walk there are heuristics and defined rules for determining the validity of traversing a specific edge in the graph to result in a full graph-traversal. One of these methods is to traverse the graph in a depth first manner (as we previously implemented in class) to avoid sectioning off any part of the graph in the traversal. In our implementation we will ignore these for simplicity.

```
eulerian_walk:
Beginning at first_node as node

For node:
    follow a random valid edge from node
    remove edge
    recurse
```


In [435]:
from collections import defaultdict
import random

class DeBruijnGraph():
    """Main class for De Bruijn graphs
    
    Private Attributes:
        graph (defaultdict of lists): Edges for De Bruijn graph
        first_node (str): starting position for traversing the graph
    """

    def __init__(self, input_string, k):
        self.graph = defaultdict(list)
        self.first_node = ''
        self.build_debruijn_graph(input_string, k)
        
    def add_edge(self, left, right):
        ''' This function adds a new edge to the graph
        
        Args:
            left (str): The k-1 mer for the left edge
            right (str): The k-1 mer for the right edge

        Updates graph attribute to add right to the list named left in defaultdict   
        '''
        self.graph[left].append(right)
        
    def remove_edge(self, left, right):
        ''' This function removed an edge from the graph
        
        Args:
            left (str): The k-1 mer for the left edge
            right (str): The k-1 mer for the right edge

        Updates graph attribute to remove right from the list named left in defaultdict
        '''
        matching_edges = []
        for i, key in enumerate(self.graph[left]):
            if key == right:
                self.graph[left].pop(i)
                break

        
    def build_debruijn_graph(self, input_string, k):
        ''' This function builds a De Buijn graph from a string
        
        Args:
            input_string (str): string to use for building the graph
            k (int): k-mer length for graph construction

        Updates graph attribute to add all valid edges from the string
        
        Example:
        >>> dbg = DeBruijnGraph("this this this is a test", 4)
        >>> print(dbg.graph) #doctest: +ELLIPSIS +NORMALIZE_WHITESPACE
        defaultdict(<class 'list'>, {'thi': ['his', 'his', 'his'], 'his': ['is ', 'is ', 'is '], ...)
        '''
        for i in range(len(input_string) - k + 1):
            kmer = input_string[i:i+k]
            left_mer = kmer[0:k-1]
            right_mer = kmer[1:k]
            self.add_edge(left_mer, right_mer)
            
            if i == 0:
                self.first_node = left_mer
        
        #self.add_edge(right_mer, self.first_node)
            
    def print_eulerian_walk(self, seed=None):
        ''' This function starts the recursive walk function
        at the first node in the graph
        
        Args: None
        
        Returns:
            tour (list): list of k-1 mers traversed by the algorithm
        
        Example:
        >>> dbg = DeBruijnGraph("this this this is a test", 4)
        >>> dbg.print_eulerian_walk(seed=1) #doctest: +ELLIPSIS +NORMALIZE_WHITESPACE
        ['thi', 'his', 'is ', 's i', ' is', 'is ', ...]
        '''
        tour = []
        tour = self.eulerian_walk(self.first_node, seed=seed)
        tour.append(self.first_node)
        return tour[::-1]
        
    def eulerian_walk(self, node, seed=None):
        ''' This is a recursive function that follows all edges from a node
        to traverse the graph
        
        Args: 
            node (str): current node to traverse from
            seed (int): seed for random selection of edge to follow
        
        Returns:
            tour (list): list of k-1 mers traversed so far by the algorithm
            Note: this will be reverse order because of recursion
            
        Example:
        >>> dbg = DeBruijnGraph("this this this is a test", 4)
        >>> dbg.eulerian_walk('thi', seed=1) #doctest: +ELLIPSIS +NORMALIZE_WHITESPACE
        ['is ', 'his', 'thi', ' th', ...]
        '''
        tour = []
        random.seed(seed)
        random.shuffle(self.graph[node])
        for next_node in self.graph[node]:
            self.remove_edge(node, next_node)
            tour = self.eulerian_walk(next_node, seed=seed)
            tour.append(next_node)
        return tour


In [434]:
graph = DeBruijnGraph("fool me once shame on shame on you fool me", 6)
print(graph.graph)
walk = graph.print_eulerian_walk(seed=11)
walk[0] + ''.join(map(lambda x: x[-1], walk[1:]))

defaultdict(<class 'list'>, {'fool ': ['ool m', 'ool m'], 'ool m': ['ol me', 'ol me'], 'ol me': ['l me '], 'l me ': [' me o'], ' me o': ['me on'], 'me on': ['e onc', 'e on ', 'e on '], 'e onc': [' once'], ' once': ['once '], 'once ': ['nce s'], 'nce s': ['ce sh'], 'ce sh': ['e sha'], 'e sha': [' sham'], ' sham': ['shame', 'shame'], 'shame': ['hame ', 'hame '], 'hame ': ['ame o', 'ame o'], 'ame o': ['me on', 'me on'], 'e on ': [' on s', ' on y'], ' on s': ['on sh'], 'on sh': ['n sha'], 'n sha': [' sham'], ' on y': ['on yo'], 'on yo': ['n you'], 'n you': [' you '], ' you ': ['you f'], 'you f': ['ou fo'], 'ou fo': ['u foo'], 'u foo': [' fool'], ' fool': ['fool ']})


'fool me once shame on shame on you fool me'

In [431]:
import doctest
doctest.testmod()

TestResults(failed=0, attempted=6)